In [54]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
from collections import Counter

## Load data

### Charger les données

In [73]:
# Load subjects
subjects = pd.read_excel('Articles_excel_vf_12_06.xlsx', sheet_name="Subjects", header=2)
# Garder les 3 premières colonnes
subjects = subjects.iloc[:, :3]

# Renommer la première colonne
subjects = subjects.rename(columns={subjects.columns[0]: "code"})
subjects.head()

,code,Descriptor,Description
0,MCAT,Commodity/Financial Market News,Markets for trading in financial instruments o...
1,M14,Commodity Markets,"Market data, such as daily quotes and trading ..."
2,M141,Agricultural Commodity Markets,"Market data, such as daily quotes and trading ..."
3,MLUMB,Forest Products Markets,"Market data, such as daily quotes and trading ..."
4,MFRUIT,Fruit Markets,"Market data, such as daily quotes and trading ..."


In [74]:
df = pd.read_excel('Articles_excel_vf_12_06.xlsx')
df.head()

,source_name,title,publication_date,subject_codes,word_count,Dossier,nom_fichier,publisher_name,source_code,industry_codes,section,document_type,region_of_origin,region_codes,an,ingestion_datetime,publication_datetime,modification_date,modification_datetime,copyright,language_code,dateline,byline
0,L'Humanité,L’œil de Bobika,2025-04-14,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744570903000_0,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250413el4e00002,2025-04-13 19:01:43,2025-04-14,2025-04-14 08:52:40,2025-04-13 19:01:43,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN
1,L'Humanité,Claude Ven est décédé le 4 avril à l’âge de 63...,2025-04-15,",gcat,gjob,",400,articles_pdf_national,LHumanité_1744657577000_1,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",britty,eecz,eurz,fra,medz,weurz,",HUMAN00020250414el4f000rx,2025-04-14 19:06:17,2025-04-15,2025-04-15 07:31:12,2025-04-14 19:06:17,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN
2,L'Humanité,L’œil de Bobika,2025-04-15,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744657578000_2,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250414el4f000s9,2025-04-14 19:06:18,2025-04-15,2025-04-15 07:33:00,2025-04-14 19:06:18,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN
3,L'Humanité,L’œil de Gros,2025-04-16,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744743866000_3,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250415el4g000m9,2025-04-15 19:04:26,2025-04-16,2025-04-16 07:30:33,2025-04-15 19:04:26,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN
4,Libération,Réponse fraternelle à Dominique de Villepin : ...,2025-04-15,",gcat,gpir,gpol,",1180,articles_pdf_national,Libération_1744806835000_4,Libération,LIBERA,NaN,NaN,article,AFRICA MEDZ MOROC NAFR,",eecz,eurz,fra,medz,weurz,",LIBERA0020250416el4f00001,2025-04-16 12:33:55,2025-04-15,2025-04-17 07:29:29,2025-04-16 12:33:55,Copyright 2025 Libération,fr,NaN,NaN


### Créer le mapping des sujets

In [75]:
# Construction du dictionnaire de correspondance (clé = code en minuscules)
mapping = dict(zip(subjects["code"].str.lower(), subjects.iloc[:, 1]))

In [76]:
mapping

{'mcat': 'Commodity/Financial Market News',
 'm14': 'Commodity Markets',
 'm141': 'Agricultural Commodity Markets',
 'mlumb': 'Forest Products Markets',
 'mfruit': 'Fruit Markets',
 'mgroil': 'Grains/Edible Oils Markets',
 'medoil': 'Edible Oils Markets',
 'mveoil': 'Vegetable Oil Markets',
 'mgrain': 'Grain Markets',
 'mcorn': 'Corn Markets',
 'mrice': 'Rice Markets',
 'mwheat': 'Wheat Markets',
 'moilsd': 'Oilseed Markets',
 'mgrfds': 'Livestock Meals/Feeds Markets',
 'mlvstk': 'Livestock/Meat Markets',
 'mcattl': 'Beef/Dairy Cattle Markets',
 'mdair': 'Dairy Markets',
 'mhogs': 'Pork/Pigs Markets',
 'mpoult': 'Poultry/Eggs Markets',
 'mwool': 'Wool Markets',
 'mpulse': 'Pulse Markets',
 'mbean': 'Beans Markets',
 'mrubb': 'Rubber Markets',
 'magric': 'Soft Commodity Markets',
 'mcocoa': 'Cocoa Markets',
 'mcoff': 'Coffee Markets',
 'mcott': 'Cotton Markets',
 'moranj': 'Orange Juice Markets',
 'msugar': 'Sugar Markets',
 'mspice': 'Spices Markets',
 'm143': 'Energy Markets',
 'mcoal

In [77]:
# Transformation de la colonne de codes en liste de significations
df["subjects"] = df["subject_codes"].apply(lambda s: [
    mapping.get(code.strip().lower(), f"[unknown:{code}]")
    for code in str(s).strip(',').split(',')
    if code.strip()
] if isinstance(s, str) else [])

In [78]:
df['subjects_sep'] = df['subjects'].apply(lambda x: ",".join(x))

### Rajouter les liens google drive

In [79]:
mapping_id = pd.read_csv('sample - Matching articles id drive.csv')

In [80]:
mapping_id.rename(columns={'Nom du fichier':'nom_fichier', 'ID':'fichier_id'}, inplace=True)

In [81]:
mapping_id['nom_fichier'] = mapping_id['nom_fichier'].apply(lambda x :x[:-4])

In [82]:
# 1. Join sur le nom de fichier pour récupérer l'ID Drive
df = df.merge(mapping_id, on="nom_fichier", how="left")

# 2. Construire le lien hypertexte
# Format Drive : https://drive.google.com/file/d/<file_id>/view
df["lien_drive"] = df["fichier_id"].apply(lambda fid: f"https://drive.google.com/file/d/{fid}/view" if pd.notnull(fid) else None)


In [83]:
df.head()

,source_name,title,publication_date,subject_codes,word_count,Dossier,nom_fichier,publisher_name,source_code,industry_codes,section,document_type,region_of_origin,region_codes,an,ingestion_datetime,publication_datetime,modification_date,modification_datetime,copyright,language_code,dateline,byline,subjects,subjects_sep,fichier_id,lien_drive
0,L'Humanité,L’œil de Bobika,2025-04-14,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744570903000_0,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250413el4e00002,2025-04-13 19:01:43,2025-04-14,2025-04-14 08:52:40,2025-04-13 19:01:43,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN,"[Content Types, Headline-Only Content]","Content Types,Headline-Only Content",1QQLMoJkwjEs4s3rLJBO_XtyKy5KWeKN4,https://drive.google.com/file/d/1QQLMoJkwjEs4s...
1,L'Humanité,Claude Ven est décédé le 4 avril à l’âge de 63...,2025-04-15,",gcat,gjob,",400,articles_pdf_national,LHumanité_1744657577000_1,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",britty,eecz,eurz,fra,medz,weurz,",HUMAN00020250414el4f000rx,2025-04-14 19:06:17,2025-04-15,2025-04-15 07:31:12,2025-04-14 19:06:17,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN,"[Political/General News, Labor Issues]","Political/General News,Labor Issues",1gmFNy4kFjxoNGllm2WhIossCC8mEqLQJ,https://drive.google.com/file/d/1gmFNy4kFjxoNG...
2,L'Humanité,L’œil de Bobika,2025-04-15,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744657578000_2,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250414el4f000s9,2025-04-14 19:06:18,2025-04-15,2025-04-15 07:33:00,2025-04-14 19:06:18,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN,"[Content Types, Headline-Only Content]","Content Types,Headline-Only Content",1Ln1OlYc77_hehzHNGGYXICF8YRV-eSA2,https://drive.google.com/file/d/1Ln1OlYc77_heh...
3,L'Humanité,L’œil de Gros,2025-04-16,",ncat,nhoc,",4,articles_pdf_national,LHumanité_1744743866000_3,Société Nouvelle du Journal l'Humanité,HUMAN,NaN,NaN,article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",HUMAN00020250415el4g000m9,2025-04-15 19:04:26,2025-04-16,2025-04-16 07:30:33,2025-04-15 19:04:26,Copyright 2025 l'Humanité All Rights Reserved,fr,NaN,NaN,"[Content Types, Headline-Only Content]","Content Types,Headline-Only Content",1ywiXNKfil14JMUgQbLHG-y224_DH_C2q,https://drive.google.com/file/d/1ywiXNKfil14JM...
4,Libération,Réponse fraternelle à Dominique de Villepin : ...,2025-04-15,",gcat,gpir,gpol,",1180,articles_pdf_national,Libération_1744806835000_4,Libération,LIBERA,NaN,NaN,article,AFRICA MEDZ MOROC NAFR,",eecz,eurz,fra,medz,weurz,",LIBERA0020250416el4f00001,2025-04-16 12:33:55,2025-04-15,2025-04-17 07:29:29,2025-04-16 12:33:55,Copyright 2025 Libération,fr,NaN,NaN,"[Political/General News, Politics/Internationa...","Political/General News,Politics/International ...",1yhxCtJGV_Y7LG-D8MCRXayAwZ90gC62G,https://drive.google.com/file/d/1yhxCtJGV_Y7LG...


### Export de données

In [84]:
df.to_csv("export_factiva.csv")

### Analyses - Obtenir le top par catégories

In [44]:
# Étape 1 : aplatir toutes les catégories
all_categories = df['subjects_sep'].dropna().apply(lambda x: [cat.strip() for cat in x.split(',')])

# Étape 2 : compter avec Counter
flat_list = [cat for sublist in all_categories for cat in sublist]
counter = Counter(flat_list)

# Étape 3 : transformer en DataFrame trié
df_counts = pd.DataFrame(counter.items(), columns=['categorie', 'nb_mentions'])
df_counts = df_counts.sort_values(by='nb_mentions', ascending=False).reset_index(drop=True)

print(df_counts)

                            categorie  nb_mentions
0              Political/General News        11539
1                       Content Types         4669
2                     Factiva Filters         4016
3    Politics/International Relations         2853
4                      [unknown:nfce]         2767
..                                ...          ...
580                  Geriatric Health            1
581                            Squash            1
582                        Meningitis            1
583                      Orienteering            1
584            Bollywood/Indian Films            1

[585 rows x 2 columns]


In [55]:
df_counts.head(200)

,categorie,nb_mentions
0,Political/General News,11539
1,Content Types,4669
2,Factiva Filters,4016
3,Politics/International Relations,2853
4,[unknown:nfce],2767
5,Domestic Politics,2623
6,Routine General News,2504
7,Society/Community,2368
8,Arts/Entertainment,2144
9,Sports,2067


# OLD

In [40]:
df.head().to_excel('sample.xlsx', index=False)

In [14]:
nom_fichier = "MlemagazineduMonde_1745578905000_284"
df.query('nom_fichier==@nom_fichier')

,source_name,title,publication_date,subject_codes,word_count,Dossier,nom_fichier,publisher_name,source_code,industry_codes,section,document_type,region_of_origin,region_codes,an,ingestion_datetime,publication_datetime,modification_date,modification_datetime,copyright,language_code,dateline,byline
284,"M, le magazine du Monde","pilier de ""Top chef"", journaliste gastronomiqu...",2025-04-26,",gcat,gfod,glife,ncat,nhoc,",21,articles_pdf_national,MlemagazineduMonde_1745578905000_284,Société Editrice du Monde,MONDMAG,NaN,", Une",article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",MONDMAG020250425el4q00006,2025-04-25 11:01:45,2025-04-26,2025-04-26 07:31:37,2025-04-25 11:01:45,(c) Le Monde. 2025. Tous droits réservés.,fr,NaN,NaN


In [12]:
result = df[df.nom_fichier == "MlemagazineduMonde_1745578905000_284"]

In [13]:
result

,source_name,title,publication_date,subject_codes,word_count,Dossier,nom_fichier,publisher_name,source_code,industry_codes,section,document_type,region_of_origin,region_codes,an,ingestion_datetime,publication_datetime,modification_date,modification_datetime,copyright,language_code,dateline,byline
284,"M, le magazine du Monde","pilier de ""Top chef"", journaliste gastronomiqu...",2025-04-26,",gcat,gfod,glife,ncat,nhoc,",21,articles_pdf_national,MlemagazineduMonde_1745578905000_284,Société Editrice du Monde,MONDMAG,NaN,", Une",article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",MONDMAG020250425el4q00006,2025-04-25 11:01:45,2025-04-26,2025-04-26 07:31:37,2025-04-25 11:01:45,(c) Le Monde. 2025. Tous droits réservés.,fr,NaN,NaN


In [15]:
result['lien'] = result['nom_fichier'].apply(
    lambda x: f'=HYPERLINK("Article national/{x}.pdf", "{x}.pdf")'
)

C:\Users\Arnaud 1\AppData\Local\Temp\ipykernel_7080\1442908695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['lien'] = result['nom_fichier'].apply(


In [16]:
result

,source_name,title,publication_date,subject_codes,word_count,Dossier,nom_fichier,publisher_name,source_code,industry_codes,section,document_type,region_of_origin,region_codes,an,ingestion_datetime,publication_datetime,modification_date,modification_datetime,copyright,language_code,dateline,byline,lien
284,"M, le magazine du Monde","pilier de ""Top chef"", journaliste gastronomiqu...",2025-04-26,",gcat,gfod,glife,ncat,nhoc,",21,articles_pdf_national,MlemagazineduMonde_1745578905000_284,Société Editrice du Monde,MONDMAG,NaN,", Une",article,EUR FRA MEDZ WEURZ,",eecz,eurz,fra,medz,weurz,",MONDMAG020250425el4q00006,2025-04-25 11:01:45,2025-04-26,2025-04-26 07:31:37,2025-04-25 11:01:45,(c) Le Monde. 2025. Tous droits réservés.,fr,NaN,NaN,"=HYPERLINK(""Article national/MlemagazineduMond..."


In [17]:
result.to_excel("liens_google_sheets.xlsx", index=False)